# 🚗 Dynamic Parking Pricing - Capstone Project

In [ ]:

!pip install pathway bokeh panel --quiet


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn


In [ ]:

df = pd.read_csv("cleaned_dataset.csv")
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values("Timestamp").reset_index(drop=True)
df.head()


In [ ]:

class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficLevel: int
    IsSpecialDay: int
    VehicleTypeWeight: float


In [ ]:

data = pw.demo.replay_csv("cleaned_dataset.csv", schema=ParkingSchema, input_rate=1000)

fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


## 💸 Model 1: Linear Occupancy-Based Pricing

In [ ]:

delta_window_1 = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber + ":" + pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(minutes=30)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ=pw.reducers.mean(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
    )
    .with_columns(
        price=10 + 5 * (pw.this.occ / pw.this.cap)
    )
)


## 📈 Model 2: Demand-Based Pricing

In [ ]:

delta_window_2 = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber + ":" + pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(minutes=30)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ=pw.reducers.mean(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        q=pw.reducers.mean(pw.this.QueueLength),
        trf=pw.reducers.mean(pw.this.TrafficLevel),
        sp=pw.reducers.max(pw.this.IsSpecialDay),
        veh=pw.reducers.mean(pw.this.VehicleTypeWeight),
    )
    .with_columns(
        demand = (
            0.5 * (pw.this.occ / pw.this.cap) +
            0.3 * pw.this.q -
            0.2 * pw.this.trf +
            0.1 * pw.this.sp +
            0.2 * pw.this.veh
        ),
        price = pw.this.demand * 5 + 10
    )
)


## 📊 Visualization (Example using Model 2)

In [ ]:

pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Dynamic Price Over Time (Model 2)",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig

viz = delta_window_2.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


In [ ]:

%%capture --no-display
pw.run()
